In [2]:
import numpy as np
import os
import cv2
from sklearn.model_selection  import train_test_split
from sklearn import linear_model
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential,load_model
import tensorflow as tf
from keras.layers import Dense,Flatten,Conv2D,MaxPooling2D
from sklearn.preprocessing import OneHotEncoder
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow as tf

In [14]:
base_size_x=32
base_size_y=32

_model=load_model ("best-accuracy-99.25-m2.h5")

def show_img(caption,img,destroy=True,show=True,wait_ms=500):
    if not show:return
    cv2.imshow(caption, img)
    cv2.waitKey(wait_ms)
    if destroy:
        cv2.destroyAllWindows()

In [10]:
def load_adad_file(adad_file,show_process=True):
    image = cv2.imread(adad_file)

    show_img("Base image", image,show=show_process)
    image_base=image

    min_ff_count=0;
    number_strting=""

    image = cv2.bitwise_not(image)
    img_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    img_bluerd=cv2.blur(img_gray,(5,10))
    show_img("Blured image", img_bluerd,show=show_process)

    _, img_bw = cv2.threshold(img_bluerd, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    show_img("Prepared image", img_bw,show=show_process)
    img_bw = cv2.rotate(img_bw, cv2.ROTATE_90_CLOCKWISE)

    show_img("Rotated image", img_bw,show=show_process)

    width=img_bw.shape[1]
    height=img_bw.shape[0]

    for x in range(height):
        text_pixel = np.where(img_bw[x,:] == 255)[0]
        if text_pixel.size > 0:
            y= (text_pixel[0])
            img_ff=img_bw.copy()
            
            ff_count, ff_image, ff_mask, ff_rect=cv2.floodFill(img_ff, None, (y, x), 64)
            ff_count, ff_image, ff_mask, ff_rect=cv2.floodFill(img_bw, None, (y, x), 0)
            show_img("Current digit", img_ff,False,show=show_process)

            img_diff=img_bw-img_ff
            img_diff[img_diff != 0] = 255
            show_img("Current digit", img_diff,False,show=show_process)

            img_digit=img_diff[ff_rect[1]:ff_rect[1]+ff_rect[3],ff_rect[0]:ff_rect[0]+ff_rect[2]]
            img_digit = cv2.rotate(img_digit, cv2.ROTATE_90_COUNTERCLOCKWISE)
            
            img_digit_resized = cv2.resize(img_digit,(base_size_x,base_size_y))
            img_digit_resized=np.array([img_digit_resized])
            img_digit_resized=img_digit_resized/255
            img_digit_resized=np.expand_dims(img_digit_resized,axis=3)
            
            out=_model.predict(img_digit_resized)
            the_no=np.argmax(out[0])
            print (the_no)
            if ff_count>min_ff_count:
                number_strting=number_strting+str(the_no)
            
            show_img("Current digit", img_digit,False,show=show_process)

    zire_adad=image_base[0:50].copy()
    np.ndarray.fill(zire_adad[0:50],255)
    image_base=np.append (image_base,zire_adad,axis=0)

    cv2.putText(image_base,"The Number : " + number_strting , (10,image_base.shape[0]-15), cv2.FONT_HERSHEY_SIMPLEX,0.75, (0,0,0), 1, cv2.LINE_AA)

    cv2.destroyAllWindows()
    show_img("Final digit", image_base,show=show_process)
    print(number_strting)
    return (number_strting,image_base)


In [18]:
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk

def open_file_dialog():
    file_types = [('All image files', '*.png;*.jpg;*.bmp'),('PNG Files', '*.png'), ('JPEG Files', '*.jpg'),('Bitmap Files', '*.bmp')]
    file_path = filedialog.askopenfilename(filetypes=file_types)
    if file_path:   
        lbl_no.config(text="Working... " )
        root.update()
        show_process=chk_var.get() == 1
        the_no,the_img=load_adad_file(file_path,show_process)
        lbl_no.config(text="The number is : "+ the_no )

        the_img = cv2.cvtColor(the_img, cv2.COLOR_BGR2RGB)
        image_pil = Image.fromarray(the_img)
        image_tk = ImageTk.PhotoImage(image_pil)
        lbl_img.config(image=image_tk)
        lbl_img.image = image_tk

root = tk.Tk()
root.title("Sobanis Complex Handwritten Persian/Arabic Numbers")
screen_width = root.winfo_screenwidth()
screen_height = root.winfo_screenheight()
window_width = 800
window_height = 600
x_coordinate = int((screen_width / 2) - (window_width / 2))
y_coordinate = int((screen_height / 2) - (window_height / 2))
root.geometry(f"{window_width}x{window_height}+{x_coordinate}+{y_coordinate}")

btn_load = tk.Button(root, text="Load File", command=open_file_dialog)
btn_load.pack(anchor="center",pady=10)

chk_var = tk.IntVar()

chk_show = tk.Checkbutton(root, text="Show steps ", variable=chk_var)
chk_show.pack()
chk_var.set(1);

lbl_no = tk.Label(root, text="")
lbl_no.pack(anchor="center",pady=10)

lbl_img = tk.Label(root)
lbl_img.pack(anchor="center",pady=10)

root.mainloop()

1/1 [==============================] - 0s 47ms/step
3
1/1 [==============================] - 0s 64ms/step
4
1/1 [==============================] - 0s 64ms/step
7
1/1 [==============================] - 0s 45ms/step
6
1/1 [==============================] - 0s 48ms/step
1
1/1 [==============================] - 0s 44ms/step
9
1/1 [==============================] - 0s 47ms/step
7
1/1 [==============================] - 0s 49ms/step
5
1/1 [==============================] - 0s 49ms/step
4
347619754
1/1 [==============================] - 0s 43ms/step
0
1/1 [==============================] - 0s 58ms/step
1
1/1 [==============================] - 0s 67ms/step
2
1/1 [==============================] - 0s 43ms/step
3
1/1 [==============================] - 0s 41ms/step
4
1/1 [==============================] - 0s 98ms/step
0
1/1 [==============================] - 0s 43ms/step
6
1/1 [==============================] - 0s 41ms/step
7
1/1 [==============================] - 0s 49ms/step
8
1/1 [=============